In [1]:
!pip install mip

     |████████████████████████████████| 48.0 MB 1.5 MB/s 


In [2]:
import pandas as pd
from mip import Model, xsum, minimize, BINARY
#load data
df = pd.read_csv("Data.csv", sep=';')
num = len(df)
weights = [[1000 for j in range(num)]for i in range(num)]
names = list(df['Place_name'])
Latitude = list(df['Latitude'])
Longitude = list(df['Longitude'])
neighbors = [[int(x) for x in arr.split(',')] for arr in df['Neighbors_indice']]
diatance = [[int(x) for x in arr.split(',')] for arr in df['Neighbor_weight']]
#create model
model = Model()
nodes = set(range(num))
#create matrix of weights
for temp_index in range(num):
    for temp_neighbor in range(len(neighbors[temp_index-1])):
        weights[temp_index-1][neighbors[temp_index-1][temp_neighbor]-1] = diatance[temp_index-1][temp_neighbor]
#fix model
x = [[model.add_var(var_type=BINARY) for j in nodes] for i in nodes]
model.objective = minimize(xsum(weights[i][j] * x[i][j] for i in nodes for j in nodes))
#show places name
print('Please select index of places.')
print('Places:')
for i in range(num):
    print(i+1, names[i])
#get index of places from user  
print('Enter starting point:')
start = int(input())-1
print('Enter destination:')
end = int(input())-1

#subject to:
for i in nodes - {start, end}:
    model += xsum(x[i][j] for j in nodes) == \
     xsum(x[j][i] for j in nodes)
model += xsum(x[start][j] for j in nodes) == xsum(x[j][start] for j in nodes) + 1
model += xsum(x[end][j] for j in nodes) == xsum(x[j][end] for j in nodes) - 1
model.optimize()
#show the answer
print('               Index              Name                Latitude               Longitude')
print('Starting point: ', start+1, '     ', names[start], '            ', Latitude[start], '         ', Longitude[start])
print('destination:    ', end+1, '     ', names[end], '            ', Latitude[end], '         ', Longitude[end])
if model.num_solutions:
    print('Best way is found:')
    print('Index in order: ', start+1, end = '')
    temp_node = start
    while(temp_node != end):
        for i in range(num):
            if(x[temp_node][i].x):
                temp_node = i
                break
        print(' ->', temp_node+1, end = '')
    print('\n')
    print('Names in order: ', names[start], end = '')
    temp_node = start
    while(temp_node != end):
        for i in range(num):
            if(x[temp_node][i].x):
                temp_node = i
                break
        print(' ->', names[temp_node], end = '')
    print('\n cost = ', model.objective_value)
    

Please select index of places.
Places:
1 UT_campus_ENG
2 UT_Enghelab
3 Imam_khomeini_hospital
4 Ferdosi_park
5 Mirzaye_shirazi_subway_station
6 Kasra_hospital
7 Hemat_subway_station
8 Mellat_park
9 Baqiatallah_hospital
10 Iranmehr-hospital
Enter starting point:
1
Enter destination:
10
               Index              Name                Latitude               Longitude
Starting point:  1       UT_campus_ENG              35.72433165           51.38847099
destination:     10       Iranmehr-hospital              35.76926283           51.44155727
Best way is found:
Index in order:  1 -> 5 -> 6 -> 7 -> 10

Names in order:  UT_campus_ENG -> Mirzaye_shirazi_subway_station -> Kasra_hospital -> Hemat_subway_station -> Iranmehr-hospital
 cost =  23.0
